## Getting Started with TensorFlow for Computer Vision



### Working with the basic building blocks of the Keras API

In [2]:
# Import the required libraries from the Keras API

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras import Input
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential

In [4]:
# Create a model using the sequential API by passing a list of layers to the Sequential constructor.
# The numbers in each layer correspond to the number of neurons or units it contains.

layers = [Dense(256, input_shape=(28 * 28 * 1,),
                activation='sigmoid'),
          Dense(128, activation='sigmoid'),
          Dense(10, activation='softmax')]
sequential_model_list = Sequential(layers)

In [5]:
# Create a model using the add() method to add one layer at a time.
# The numbers in each layer correspond to the number of neurons or units it contains:

sequential_model = Sequential()
sequential_model.add(Dense(256, input_shape=(28 * 28 * 1,),
                activation='sigmoid'))
sequential_model.add(Dense(128, activation='sigmoid'))
sequential_model.add(Dense(10, activation='softmax'))

In [ ]:
# Create a model using the Functional API.
# The numbers in each layer correspond to the number of neurons or units it contains:

imput_layer = Input(shape=(28 * 28 * 1,))
dense_1 = Dense(256, activation='sigmoid')(input_layer)
